## Build a convolutional neural network (CNN) for MNIST using Keras
Convolutional neural network (NN) of Keras on MNIST data
* Author: Gao Yang  
* Note:
   * You could run this notebook on local CPU Macbook, but the convolution-heavy operations can be quite slow.
   * 2 minutes per epoch on 2.4G Intel i5 CPU (Macbook Pro)
   * After 12 epochs, test accuracy = 0.9912

#### Setup the enviorment

In [18]:
from __future__ import division, print_function
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras import backend as K

#### Prepare MNIST dataset

In [11]:
# ------ import MNIST from local mnist.npz -------
import os
def load_mnistdata(path_npz):
    assert os.path.isfile(path_npz)
    data = np.load(path_npz)
    x_train, y_train = data['x_train'], data['y_train']
    x_test, y_test = data['x_test'], data['y_test']
    return (x_train,y_train),(x_test,y_test)

path_npz = '/Users/Yang/Projects/keras-examples/mnist.npz'
(x_train,y_train),(x_test,y_test) = load_mnistdata(path_npz)

# size of each image in the dataset
img_size_rows, img_size_cols = x_train.shape[1], x_train.shape[2]

# ---- reshape the dataset according to the position of color channel required by the backend (K)

if K.image_data_format() == 'channel_first':
    x_train = x_train.reshape(x_train.shape[0],1,img_size_rows,img_size_cols)
    x_test = x_test.reshape(x_test.shape[0],1,img_size_rows,img_size_cols)
    input_shape = (1,img_size_rows,img_size_cols)
else:
    x_train = x_train.reshape(x_train.shape[0],img_size_rows,img_size_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_size_rows,img_size_cols,1)
    input_shape = (img_size_rows,img_size_cols,1)

# set pixel value to [0,1]
x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.astype('float32')
x_test /= 255

print('Image data format of K:',K.image_data_format())
print('x_train shape:', x_train.shape)
print('Train sample number: {}'.format(x_train.shape[0]))
print('Test sample number: {}'.format(x_test.shape[0]))

# convert labels to binary form
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


Image data format of K: channels_last
x_train shape: (60000, 28, 28, 1)
Train sample number: 60000
Test sample number: 10000


In [7]:
x_train.shape

(60000, 28, 28)

#### Setup the CNN model

In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
             optimizer='Adadelta',
             metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

#### Train the model

In [15]:
num_epoch = 12
batch_size = 128

model.fit(x_train,y_train,
          epochs=num_epoch,
          verbose=1,
          batch_size=batch_size,
          validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 142s 2ms/step - loss: 0.1472 - acc: 0.9558 - val_loss: 0.0692 - val_acc: 0.9778
Epoch 2/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0697 - acc: 0.9788 - val_loss: 0.0373 - val_acc: 0.9875
Epoch 3/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0536 - acc: 0.9830 - val_loss: 0.0371 - val_acc: 0.9879
Epoch 4/12
60000/60000 [==============================] - 115s 2ms/step - loss: 0.0437 - acc: 0.9861 - val_loss: 0.0307 - val_acc: 0.9897
Epoch 5/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0358 - acc: 0.9884 - val_loss: 0.0285 - val_acc: 0.9905
Epoch 6/12
60000/60000 [==============================] - 117s 2ms/step - loss: 0.0309 - acc: 0.9902 - val_loss: 0.0313 - val_acc: 0.9904
Epoch 7/12
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0282 - acc: 0.9910 - val_loss: 0.0344 - 

#### Evaluate the model

In [17]:
score = model.evaluate(x_test,y_test,verbose=1)
print('Test Loss: {:.06f}'.format(score[0]))
print('Test Accuracy: {:.06f}'.format(score[1]))

10000/10000 [==============================] - 6s 556us/step
Test Loss: 0.029685
Test Accuracy: 0.991200
